In [45]:
import sklearn
import pandas as pd
import numpy as np

In [46]:
data = pd.read_csv('SMSSpamCollection.txt', sep='\t', header=-1, dtype=str)

In [47]:
print data.columns

Int64Index([0, 1], dtype='int64')


In [48]:
list_marks = list()

for i in xrange(len(data[0])):
    if (data[0][i] == 'ham'):
        list_marks.append(0)
    else:
        list_marks.append(1)
        
list_phrases = data[1].values.tolist()

In [49]:
print len(list_marks)
print type(list_marks)
print
print len(list_phrases)
print type(list_phrases)

5572
<type 'list'>

5572
<type 'list'>


In [50]:
import sklearn.feature_extraction
from sklearn import svm

count_vectorizer = sklearn.feature_extraction.text.CountVectorizer()
features = count_vectorizer.fit_transform(list_phrases)

In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, list_marks)
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)

0.97559224694903091

In [52]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

scores = cross_val_score(clf, X_train, y_train, cv=10, scoring='f1')

In [53]:
print scores
print np.average(scores)

[ 0.90196078  0.87037037  0.99099099  0.98181818  0.90384615  0.93577982
  0.92307692  0.94230769  0.96226415  0.97196262]
0.9384377681


In [54]:
X1_test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
           "FreeMsg: Txt: claim your reward of3 hours talk time",
           "Have you visited the last lecture on physics?",
           "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
           "Only 99$"]
print X1_test

['FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB', 'FreeMsg: Txt: claim your reward of3 hours talk time', 'Have you visited the last lecture on physics?', 'Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$', 'Only 99$']


In [55]:
from sklearn.linear_model import LogisticRegression

logReg = LogisticRegression()
logReg.fit(features, list_marks)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### Прогнозы классификатора

In [56]:
print logReg.predict(count_vectorizer.transform(X1_test))

[1 1 0 0 0]


In [57]:
def diffParamsForCountVectorizer(my_ngram_range):
    my_count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=my_ngram_range)
    my_features = my_count_vectorizer.fit_transform(list_phrases)
    
    my_logReg = LogisticRegression()
    my_scores = cross_val_score(my_logReg, my_features, list_marks, cv=10, scoring='f1_macro')
    
    return np.average(my_scores)

#### Значения f1-меры

In [58]:
print round(diffParamsForCountVectorizer((2, 2)), 2), round(diffParamsForCountVectorizer((3, 3)), 2), round(diffParamsForCountVectorizer((1, 3)), 2)

0.9 0.85 0.96


In [59]:
from sklearn.naive_bayes import MultinomialNB

def diffParamsForCountVectorizerWithMulti(my_ngram_range):
    my_count_vectorizer = sklearn.feature_extraction.text.CountVectorizer(ngram_range=my_ngram_range)
    my_features = my_count_vectorizer.fit_transform(list_phrases)
    
    my_Multinominal = MultinomialNB()
    my_scores = cross_val_score(my_Multinominal, my_features, list_marks, cv=10, scoring='f1_macro')
    
    return np.average(my_scores)

#### Наивный Байес действительно страдает от нехватки статистики по биаграммам и триаграммам

In [60]:
print round(diffParamsForCountVectorizerWithMulti((2, 2)), 2), round(diffParamsForCountVectorizerWithMulti((3, 3)), 2), round(diffParamsForCountVectorizerWithMulti((1, 3)), 2)

0.78 0.52 0.93


In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
my_features = tfidf_vectorizer.fit_transform(list_phrases)
my_logReg = LogisticRegression()

scores = cross_val_score(my_logReg, my_features, list_marks, cv=10, scoring='f1_macro')

print np.average(scores)

0.916614907888


Качество понизилось

#### Вывод

Все вместе: биаграммы, триаграммы, униграммы - работает хорошо.
LogisticRegression лучше MultinominalNB